In [10]:
import numpy as np
from numpy.polynomial import Polynomial, Laguerre
import scipy.stats as stats
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
import QuantLib as ql
from multiprocess import Pool
from tqdm import tqdm
import matplotlib.pyplot as plt

## Defining Yield Curve

In [11]:
# Flat Yield Curve
flat_yields = [
    (0.5, .04),
    (2.0, .04),
    (5.0, .04),
    (10.0, .04),
]

def create_yield_curve(yield_curve):
    # Extract durations and yields
    durations = [item[0] for item in yield_curve]
    yields = [item[1] for item in yield_curve]

    # Fit cubic spline
    yield_curve = CubicSpline(durations, yields)
    return yield_curve

yield_curve = create_yield_curve(flat_yields)

## Setting Up MC Simulation

In [12]:
def get_short_rates(simulations, steps, dt, r0, a, sigma, theta, yield_curve):
    def simulate_path(params):

        _, Z, steps, dt, r0, a, sigma, theta, yield_curve = params

        import numpy as np

        theta = lambda t: yield_curve.derivative(nu=1)(t) + (a*yield_curve(t)) + ((sigma**2)/(2*a))*(1-np.exp(-2*a*t))
        dr = np.zeros(steps)
        r = np.zeros(steps)
        r[0] = r0  # Initialize with the initial rate from the market
        for t in range(1, steps):
            dr[t] = (theta(t * dt) - a * r[t-1]) * dt + sigma * np.sqrt(dt) * Z[t]
            r[t] = r[t-1] + dr[t]
        return r

    # Use multiprocess to run simulations in parallel
    # Generating all the random numbers at the same moment to reduce variance + Using Antitheti Variables
    random_part_1 = np.random.normal(0, 1, size=(int(simulations/2), steps))    
    random_part_2 = -random_part_1
    random_z = np.vstack((random_part_1, random_part_2))
    simulation_params = [(i, random_z[i,:], steps, dt, r0, a, sigma, theta, yield_curve) for i in range(simulations)]

    # Run in Parallel
    with Pool() as pool:
        short_rate_paths = pool.map(simulate_path, simulation_params)

    return np.array(short_rate_paths)

In [13]:
def get_forward_rates(simulations, months_per_year, dt, T_m, tenor, short_rate_paths):
    # Finding forward rates based on the simulated short rates
    forward_rate_paths_regression = []
    for simulation in range(simulations):
        forward_rates_temp = []
        for t in np.arange(0.0, T_m, tenor):
            # To be clear what is counted where, made variables as explicit as possible
            start_id = int(months_per_year*t)
            finish_id = int(months_per_year*(t+tenor))
            forward_rate = np.sum(short_rate_paths[simulation, start_id:finish_id]*dt)  # Using Reimann sum to approximate the integral for rates for correct time period
            forward_rates_temp.append(forward_rate)
        forward_rate_paths_regression.append(forward_rates_temp)
    return (np.array(forward_rate_paths_regression))         # discounting value for each period between tenors

In [14]:
def get_simulations_swap_values(simulations, months_per_year, T_m, tenor, N, fixed_rate, short_rate_paths, yield_curve, a, sigma):
    # Swap Values calculator
    def sim_swap_eval(params):
        _, months_per_year, T_m, tenor, N, fixed_rate, short_rate_path, yield_curve, a, sigma = params

        import numpy as np

        # Bond Evaluation
        def P(s, t, r_s, a, sigma, yield_curve):

            def A(s, t, a, sigma, yield_curve):   
                P_0_t = np.exp(-yield_curve(t)*t)     # Value of Zero Coupon Bond (0,t)
                P_0_s = np.exp(-yield_curve(s)*s)     # Value of Zero Coupon Bond (0,s)
                term1 = P_0_t/P_0_s                 
                term2 = B(s, t, a) * yield_curve(s) # B * Instantenious Forward Rate
                term3 = (sigma**2/(4*a)) * B(s,t,a)**2 * (1-np.exp(-2*a*s))
                return term1*np.exp(term2-term3)

            def B(s, t, a):
                return 1/a * (1 - np.exp(a * (s - t)))
                
            return A(s, t, a, sigma, yield_curve) * np.exp(-B(s, t, a) * r_s)     # Short rate and initial rate are the same here
        # Swap Evaluation
        def V(months_per_year, t, T_0, T_m, N, fixed_rate, tenor, rates, a, sigma, yield_curve):
            term1 = P(t, T_0, rates[int(months_per_year*t)], a, sigma, yield_curve)
            term2 = P(t, T_m, rates[int(months_per_year*t)], a, sigma, yield_curve)
            term3 = 0
            for T_i in np.arange(T_0+tenor, T_m+tenor, tenor):       # For all intermediate and last one payout dates (np.arange does not include m)
                term3 += P(t, T_i, rates[int(months_per_year*t)],  a, sigma, yield_curve) * tenor
            return -N*(term1 - term2 - fixed_rate*term3)     # (we are buying swaption) Only put makes sense, otherwise pointless to exercise early

        # Finding Swap Value for each possible exercise point
        swap_values = []
        for t in np.arange(tenor, T_m, tenor):     # Because of delay on starting of payments, we have less entries (Last one must be a tenor before Maturity)
            swap_values.append(V(months_per_year, t, t, T_m, N, fixed_rate, tenor, short_rate_path, a, sigma, yield_curve))
        return (swap_values)

    # Use multiprocess to run simulations in parallel
    simulation_params = [(i, months_per_year, T_m, tenor, N, fixed_rate, short_rate_paths[i], yield_curve, a, sigma) for i in range(simulations)]
    # Run in Parallel
    with Pool() as pool:
        simulations_swap_values = pool.map(sim_swap_eval, simulation_params)

    return np.array(simulations_swap_values)

## Calculating European Swaption Value

In [15]:
def get_fair_value_ql(T_m, tenor, a, sigma, fixed_rate, r0):
    # Setup the market and yield term structure
    calendar = ql.TARGET()
    day_count = ql.Actual365Fixed()
    todays_date = ql.Date(1, 1, 2023)
    ql.Settings.instance().evaluationDate = todays_date
    flat_forward = ql.FlatForward(todays_date, r0, day_count)     # Flat rate curve
    yield_curve_handle = ql.YieldTermStructureHandle(flat_forward)

    # Swaption characteristics
    # Define the fixed-rate leg
    start_date = todays_date
    maturity_date = todays_date + ql.Period(int(T_m*12), ql.Months)
    fixed_leg_tenor = ql.Period(int(tenor*12), ql.Months)
    fixed_leg_schedule = ql.Schedule(start_date, maturity_date, fixed_leg_tenor, calendar,
                                    ql.ModifiedFollowing, ql.ModifiedFollowing,
                                    ql.DateGeneration.Forward, False)

    # Define the floating-rate leg
    index = ql.IborIndex("CustomEuriborM", ql.Period(int(tenor*12), ql.Months), 0, ql.EURCurrency(), calendar, ql.ModifiedFollowing, False, day_count, yield_curve_handle)
    floating_leg_schedule = ql.Schedule(start_date, maturity_date, index.tenor(), calendar,
                                        ql.ModifiedFollowing, ql.ModifiedFollowing,
                                        ql.DateGeneration.Forward, False)

    # Define the swaption
    hull_white_model = ql.HullWhite(yield_curve_handle, a, sigma)

    # Setup the pricing engine
    engine = ql.TreeSwaptionEngine(hull_white_model, 200)

    # # Define the European swaption    (Outputs same value as swap)
    exercise = ql.EuropeanExercise(todays_date + ql.Period(int(tenor*12), ql.Months))       # Very small period is necessary, otherwise output is zero

    european_swaption = ql.Swaption(ql.VanillaSwap(ql.VanillaSwap.Receiver, 1.0, fixed_leg_schedule,
                                        fixed_rate, day_count, floating_leg_schedule,
                                        index, 0.0, index.dayCounter()), exercise)
    
    # Setup the pricing engine for European swaption
    european_swaption.setPricingEngine(engine)
    european_swaption_npv = european_swaption.NPV()
    # print("QuantLib European Swaption price is: ", european_swaption_npv)
    return european_swaption_npv

In [16]:
def price_european_MC(swap_values, T_n, forward_rates):
    positive_swaps = list()
    for i in range(len(swap_values)):
        if swap_values[i] > 0:
            disc_rate = np.sum(forward_rates[i, :int(T_n)+1])
            positive_swaps.append(swap_values[i]* np.exp(- disc_rate))
        else:
            positive_swaps.append(0)
    return sum(positive_swaps) / len(swap_values)

## Checking Correctness of Results

In [17]:
# Parameters
T = 10                      # Years of Simulation
months_per_year = 12         # Days in the Year
steps = int(T*months_per_year)    # Number of steps
dt = 1.0 * T / steps            # Time step size
simulations = 20_000        # Amount of Monte Carlo runs
r0 = 0.04

# Parameters for the Hull-White model
a = 0.01
sigma = 0.01
theta_normal = lambda t: yield_curve.derivative(nu=1)(t) + (a*yield_curve(t)) + ((sigma**2)/(2*a))*(1-np.exp(-2*a*t))

# Swaption parameters
T_m = 10.0              # Maximum duration of swaption (Maturity)
tenor = 0.5            # Semi-Annual settlements (First settlement is assumed to be possible after this period too)
N = 1                   # Notional Amount
fixed_rate = 0.04

In [18]:
# Simulate Monte Carlo Paths
normal_rates = get_short_rates(simulations, steps, dt, r0, a, sigma, theta_normal, yield_curve)
forward_rate_normal = get_forward_rates(simulations, months_per_year, dt, T_m, tenor, normal_rates)

fixed_rates = np.linspace(0.032, 0.048, 5)
for strike_rate in fixed_rates:
    swap_values = get_simulations_swap_values(simulations, months_per_year, T_m, tenor, N, strike_rate, normal_rates, yield_curve, a, sigma)[:,0]
    sim_value = price_european_MC(swap_values, tenor, forward_rate_normal)

    ql_value = get_fair_value_ql(T_m, tenor, a, sigma, strike_rate, r0)

    print("Fixed Rate = ", strike_rate)
    print("Absolute Difference", np.abs(sim_value - ql_value))
    print("Relative Difference", np.abs(sim_value - ql_value) / ql_value)
    print("\n")

Fixed Rate =  0.032
Absolute Difference 1.218966704170368e-05
Relative Difference 0.0043262034853047355


Fixed Rate =  0.036000000000000004
Absolute Difference 9.66639488145047e-05
Relative Difference 0.011690649382319052


Fixed Rate =  0.04
Absolute Difference 0.0002506586210402513
Relative Difference 0.012921263184743684


Fixed Rate =  0.044
Absolute Difference 0.00026238291635830024
Relative Difference 0.006999845495693887


Fixed Rate =  0.048
Absolute Difference 0.00012061594205124704
Relative Difference 0.001953533452585574


